<a href="https://colab.research.google.com/github/bilalProgTech/online-data-science-ml-challenges/blob/master/Machine-Hack-Music-Genre-ML-Hackathon/Benchmark-Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MachineHack-Music-Genre-ML-Hackathon

[Hackathon](https://machinehack.com/hackathonsmusic_genre_classification_weekend_hackathon_edition_2_the_last_hacker_standing/overview)

Accuracy: 1.17052

In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/music_genre_classification_weekend_hackathon_edition_2_the_last_hacker_standing/Dataset_MGP_LHS.zip
!unzip *.zip

--2021-08-12 12:34:13--  https://machinehack-be.s3.amazonaws.com/music_genre_classification_weekend_hackathon_edition_2_the_last_hacker_standing/Dataset_MGP_LHS.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.62.68
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.62.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1328647 (1.3M) [application/octet-stream]
Saving to: ‘Dataset_MGP_LHS.zip.1’

Dataset_MGP_LHS.zip 100%[===================>]   1.27M   867KB/s    in 1.5s    

2021-08-12 12:34:16 (867 KB/s) - ‘Dataset_MGP_LHS.zip.1’ saved [1328647/1328647]

Archive:  Dataset_MGP_LHS.zip
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [ ]:
train.isnull().sum()

Artist Name              0
Track Name               0
Popularity             428
danceability             0
energy                   0
key                   2014
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      4377
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64

In [ ]:
test = pd.read_csv('test.csv')
test.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature
0,David Bowie,Space Oddity - 2015 Remaster,73.0,0.310,0.403,NaN,-13.664,1,0.0326,0.07260,0.000093,0.1390,0.466,134.480,318027.000000,4
1,Crimson Sun,Essence of Creation,34.0,0.511,0.955,1.0,-5.059,1,0.1290,0.00040,0.000009,0.2630,0.291,151.937,220413.000000,4
2,P!nk,Raise Your Glass,78.0,0.700,0.709,7.0,-5.006,1,0.0839,0.00480,NaN,0.0289,0.625,122.019,202960.000000,4
3,Shawn Mendes,Wonder,80.0,0.333,0.637,1.0,-4.904,0,0.0581,0.13100,0.000018,0.1490,0.132,139.898,172693.000000,4
4,Backstreet Boys,Helpless When She Smiles - Radio Version,48.0,0.393,0.849,11.0,-4.114,1,0.0459,0.00421,NaN,0.1620,0.222,74.028,4.093117,4


In [ ]:
combine = train.append(test).reset_index(drop=True)
combine.shape

(25709, 17)

In [ ]:
len(combine['Artist Name'].value_counts())

11304

In [ ]:
len(combine['Track Name'].value_counts())

20303

In [ ]:
combine.isnull().sum()

Artist Name              0
Track Name               0
Popularity             655
danceability             0
energy                   0
key                   2822
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      6286
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                 7713
dtype: int64

In [ ]:
combine['Popularity'] = combine['Popularity'].fillna(combine['Popularity'].mean())
combine['key'] = combine['key'].fillna(combine['key'].mean())
combine['instrumentalness'] = combine['instrumentalness'].fillna(combine['instrumentalness'].mean())
combine.isnull().sum()

Artist Name              0
Track Name               0
Popularity               0
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                 7713
dtype: int64

In [ ]:
X = combine[combine['Class'].isnull()!=True].drop(['Artist Name','Class', 'Track Name'], axis=1)
y = combine[combine['Class'].isnull()!=True]['Class']

X_test = combine[combine['Class'].isnull()==True].drop(['Artist Name','Class','Track Name'], axis=1)

X.shape, y.shape, X_test.shape

((17996, 14), (17996,), (7713, 14))

In [ ]:
X.head()

,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature
0,60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,0.175318,0.0849,0.8990,134.071,234596.0,4
1,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4
2,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4
3,66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,0.175318,0.1220,0.5690,107.033,173968.0,4
4,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = pd.DataFrame(scaler.transform(X), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

pca = PCA()
pca.fit(X_scaled)
X_pca = pd.DataFrame(pca.transform(X_scaled), columns=X.columns)
X_test_pca = pd.DataFrame(pca.transform(X_test_scaled), columns=X.columns)
X_pca.head()

,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature
0,-0.997206,-1.998830,-1.571210,-0.433952,-0.540153,0.510491,-0.654045,1.152930,0.166838,0.813813,1.113612,-0.517072,-0.019694,0.732486
1,-0.943177,0.420795,-1.454084,-0.800596,-0.453014,-0.061934,-0.029477,0.252224,-0.062056,-0.193199,0.039670,0.980824,0.401513,-0.033561
2,0.141613,-0.166910,-0.894727,1.949031,0.552344,-0.938632,-0.097618,-0.346452,-0.409010,-0.252278,0.565122,0.662528,-0.237178,0.188058
3,-0.554276,-1.886353,1.317788,-1.409616,0.848774,-0.289510,0.188420,-0.233221,-0.121320,0.757798,-0.025536,-0.920230,0.347468,0.254405
4,-1.922611,2.514625,-1.445811,0.520995,0.051411,2.589867,-0.628155,-1.299180,-0.589005,-1.268622,-0.194465,0.431032,-0.065408,-0.201215


In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
err = []
y_pred_tot_lgm = []

from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
i = 1
for train_index, test_index in fold.split(X_scaled, y):
    x_train, x_val = X_scaled.iloc[train_index], X_scaled.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = XGBClassifier(max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994,
                       objective='multiclass')
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          verbose=200)
    pred_y = m.predict_proba(x_val)
    print(i, " err_lgm: ", log_loss(y_val, pred_y))
    err.append(log_loss(y_val, pred_y))
    pred_test = m.predict_proba(X_test_scaled)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

[0]	validation_0-merror:0.55564	validation_1-merror:0.607778
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 200 rounds.
[200]	validation_0-merror:0.342595	validation_1-merror:0.461111
[400]	validation_0-merror:0.284523	validation_1-merror:0.458889
[600]	validation_0-merror:0.240761	validation_1-merror:0.460833
Stopping. Best iteration:
[538]	validation_0-merror:0.253821	validation_1-merror:0.455833

1  err_lgm:  1.204842333714777
[0]	validation_0-merror:0.558866	validation_1-merror:0.579605
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 200 rounds.
[200]	validation_0-merror:0.33889	validation_1-merror:0.44818
[400]	validation_0-merror:0.282281	validation_1-merror:0.446235
Stopping. Best iteration:
[329]	validation_0-merror:0.301591	validation_1-merror:0.443734

2  err_lgm:  1.181

In [ ]:
submission = pd.DataFrame(np.mean(y_pred_tot_lgm, 0))
submission.to_csv('LGBM_Scaled.csv', index=False, header=True)
submission.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.000194,0.031815,0.008993,0.000015,0.000343,0.004687,0.118454,0.000010,0.051114,0.017044,0.767332
1,0.000016,0.052050,0.008122,0.000011,0.000013,0.002113,0.068320,0.000005,0.570575,0.010233,0.288542
2,0.000017,0.040280,0.000902,0.000008,0.000257,0.035672,0.024895,0.000005,0.005778,0.694541,0.197644
3,0.000083,0.057586,0.001772,0.000016,0.000048,0.066915,0.031296,0.000008,0.015827,0.542229,0.284220
4,0.052638,0.002424,0.002726,0.001374,0.095453,0.001741,0.003229,0.000248,0.013265,0.587774,0.239128
